In [1]:
import time
import qrng
import numpy as np
import pandas as pd
from math import pi

In [2]:
#################### Initialize HyperParameter ####################
MaxNsol = 300           # define MaxNsol
MaxNvar = 150           # define MaxNvar

# Nrun = 10             # define Nrun
Ngen = 30               # define Ngen
Nsol = 5                # define Nsol
Nvar = 6                # define Nvar
Nbit = 10                # define the variable bits
Qbest_sol = Nsol + 1    # solution Number include Qbest_sol
Qbest_idx = Nsol        # Index of Qbest_sol
theta_1 = 0.08 * pi
theta_2 = 0.001 * pi

Cg = 0.7                # define Cg
Cp = 0.9                # define Cp
# Cw = 0.9              # define Cw

data_lookup = {"xi": [0, 0, 0, 0, 1, 1, 1, 1],
                "bi": [0, 0, 1, 1, 0, 0, 1, 1],
                "f(x) > f(b)": [False, True, False, True, False, True, False, True],
        "theta": [0.001*pi, 0.001*pi, 0.08*pi, 0.001*pi, 0.08*pi, 0.001*pi, 0.001*pi, 0.001*pi],
        "ab > 0": [-1, -1, -1, -1, 1, 1, 1, 1],
        "ab < 0":[1, 1, 1, 1, -1, -1, -1, -1],
        "a = 0":[1, 1, 1, 1, 1, 1, 1, 1],
        "b = 0":[1, 1, 1, 1, 1, 1, 1, 1]}
#################### Initialize HyperParameter ####################
Q_lookup = pd.DataFrame(data_lookup)
Q_lookup

,xi,bi,f(x) > f(b),theta,ab > 0,ab < 0,a = 0,b = 0
0,0,0,False,0.003142,-1,1,1,1
1,0,0,True,0.003142,-1,1,1,1
2,0,1,False,0.251327,-1,1,1,1
3,0,1,True,0.003142,-1,1,1,1
4,1,0,False,0.251327,1,-1,1,1
5,1,0,True,0.003142,1,-1,1,1
6,1,1,False,0.003142,1,-1,1,1
7,1,1,True,0.003142,1,-1,1,1


In [3]:

def Q_lookup(comp_cond, xi, bi, ab_cond):
    '''
    Q_lookup(f(x) > f(b)? , xi, bi, ab)
    '''
    Q_lookup = pd.DataFrame(data_lookup)
    theta = Q_lookup.loc[Q_lookup["f(x) > f(b)"] == comp_cond]
    theta = theta.loc[theta["xi"] == xi]
    theta = theta.loc[theta["bi"] == bi]
    if ab_cond == 0:
        return theta["theta"].values[0] * theta["ab > 0"].values[0]
    if ab_cond == 1:
        return theta["theta"].values[0] * theta["ab < 0"].values[0]
    if ab_cond == 2:
        return theta["theta"].values[0] * theta["a = 0"].values[0]
    # if ab_cond == 3:
    #     return theta["theta"].values[0] * theta["b = 0"].values[0]
    else:
        print("Q_lookup ERROR!!")


In [4]:
def ab_test(a, b):
    if (a * b) > 0: 
        return 0
    elif (a * b) < 0:
        return 1
    elif a == 0 or b == 0:
        return 2
    # elif b == 0:
    #     return 3
    else:
        print("ab_cond ERROR!!")

In [5]:
def Qsol_init(Nvar = Nvar, Nbit = Nbit):
    '''
    return a qc_list represent the solution value [x1, x2, x3, ...]
    arr[qvar][qubit][alpha: 0, beta: 1]
    '''
    q_list = [[[0.5, 0.5] for i in range(Nbit)] for j in range(Nvar)]
    q_list = np.sqrt(np.array(q_list))
    return q_list

In [6]:
def get_Qrnd():
    '''
    Function to get a random floating number from a Qubit in Quantum Computer.
    '''
    qrng.set_backend()
    rnd = qrng.get_random_float(0, 1)
    return rnd

In [7]:
def var_measure(qvar, representation = 10):
    '''
    var_measure function：Measure a Quantum state var to classical var 
    Second parameter can chose binary(Input = 2) representation or Decimal(Input: 10)
    '''
    var_bin = ""
    for qubit in qvar:
        rnd = get_Qrnd()
        if rnd <= round(np.square(qubit[0]),10):
            var_bin += "0"
        else:
            var_bin += "1"
    if representation == 2:
        return int(var_bin)
    elif representation == 10:
        return int(var_bin, 2)
    elif representation == "str":
        return var_bin
    else:
        return False

In [8]:
def sol_measure(qlist, representation = 10):
    '''
    Measure a solution list with Nvar: [x1, x2, x3, ...]
    from quantum representation to classical representation
    '''
    clist = []
    if representation == 10:
        for var in qlist:
            clist.append(var_measure(var))
    else:
        for var in qlist:
            clist.append(var_measure(var,2))
    return clist

In [9]:
def best_dict_init():
    '''
    Use best_dict to storage the gbest_fitness, gbest_list, pbest_fitness, pbest_list
    '''
    best_dict = {
        "gbest_fitness": 0,
        "gbest_sol": [0 for i in range(Nvar)],
        "pbest_fitness":[0 for i in range(Nsol)],
        "pbest_list":[[0 for i in range(Nvar)] for j in range(Nsol)]
    }
    return best_dict

In [10]:
def God_init():
    '''
    Initialize one generation with Nsol + 1.
    Last one qc_list is Qgbest.
    God_list[sol][var][qubit(alpha, beta)]
    '''
    return [Qsol_init(Nvar, Nbit) for sol in range(Qbest_sol)]

In [11]:
def rotate(sol, var, bit, theta):
    '''
    Operate quantum rotation gate on input qubit by theta
    '''
    global God_list
    theta = np.radians(theta)

    r = np.array(((np.cos(theta), -np.sin(theta)),
               (np.sin(theta),  np.cos(theta))))
    God_list[Qbest_idx][var][bit] = r.dot(God_list[Qbest_idx][var][bit])
    return God_list

In [12]:
def dec_to_bin(var, Nbit = Nbit):
    '''
    Define a function to convert integer to binary_string.
    In order to compare the Global best solution with current solution for each bit.
    And then can apply the rotation gate on God_list
    '''
    return format(var, f'0{Nbit}b')

In [13]:
def sol_update(sol):
    '''
    Update a solution with Nvar from Qsol_list.
    '''
    Random_Qvar = np.sqrt(np.array([[0.5, 0.5] for i in range(Nbit)]))
    sol_list = []
    for var in range(Nvar):
        rnd = get_Qrnd()
        if (rnd < Cg):
            # 從God_list中的Qgbest solution 測量第 var個解的值
            # print("Measure from Qbest")
            variable = (var_measure(God_list[Qbest_idx][var]))
            sol_list.append(variable)
        elif (rnd < Cp):
            # 從God_list中的Qpbest solution 測量第 var個解的值
            # print("Measure from Qpbest")
            variable = (var_measure(God_list[sol][var]))
            sol_list.append(variable)
        else:
            # print("Measure from Qrnd")
            variable = (var_measure(Random_Qvar))
            sol_list.append(variable)
    return sol_list

In [14]:
def get_bin_sol(sol_list):
    '''
    Convert a decimal solution list into a binary solution list(each variable is a "001101" string).
    '''
    return [dec_to_bin(var) for var in sol_list]

In [15]:
# def God_update(sol, sol_list, sol_fitness):
#     '''
#     Update the gbest and pbest for "solution_i" in best_dict.
#     '''
#     global best_dict
#     global God_list

#     # Get the current solution in binary representation
#     # In order to compare with the target quantum state
#     curr_bin_sol = get_bin_sol(sol_list)
#     gbest_bin_sol = get_bin_sol(best_dict["gbest_sol"])
#     pbest_bin_sol = get_bin_sol(best_dict["pbest_list"][sol])

#     for var in range(Nvar):
#         for bit in range(Nbit):
#             # alpha, beta
#             a_gbest = God_list[Qbest_idx][var][bit][0]
#             b_gbest = God_list[Qbest_idx][var][bit][1]
#             a_pbest = God_list[sol][var][bit][0]
#             b_pbest = God_list[sol][var][bit][1]

#             # xi: bit of current solution
#             # bi: bit of gbest/pbest solution
#             xi = int(curr_bin_sol[var][bit])
#             bi_gbest = int(gbest_bin_sol[var][bit])
#             bi_pbest = int(pbest_bin_sol[var][bit])

#             if (sol_fitness >= best_dict["gbest_fitness"]):
#                 ab_cond = ab_test(a_gbest, b_gbest)
#                 delta_theta = Q_lookup(True, xi, bi_gbest, ab_cond)
#                 # rotate(sol, var, bit, theta)
#                 rotate(Qbest_idx, var, bit, delta_theta)
#                 best_dict["gbest_fitness"] = sol_fitness
#                 best_dict["gbest_sol"] = sol_list
    
#             elif (sol_fitness < best_dict["gbest_fitness"]):
#                 ab_cond = ab_test(a_gbest, b_gbest)
#                 delta_theta = Q_lookup(False, xi, bi_gbest, ab_cond)
#                 rotate(Qbest_idx, var, bit, delta_theta)

#             if (sol_fitness >= best_dict["pbest_fitness"][sol]):
#                 ab_cond = ab_test(a_pbest, b_pbest)
#                 delta_theta = Q_lookup(True, xi, bi_gbest, ab_cond)
#                 rotate(sol, var, bit, delta_theta)
#                 best_dict["pbest_fitness"][sol] = sol_fitness
#                 best_dict["pbest_list"][sol] = sol_list
            
#             elif (sol_fitness < best_dict["pbest_fitness"][sol]):
#                 ab_cond = ab_test(a_pbest, b_pbest)
#                 delta_theta = Q_lookup(False, xi, bi_pbest, ab_cond)
#                 rotate(sol, var, bit, delta_theta)
#         print(np.square(God_list[Qbest_idx]))
#     return God_list

In [22]:
def God_update(sol, sol_list, sol_fitness):
    '''
    This function update the gbest and pbest for "SOLUTION_i" recorded in best_dict.
    '''
    global best_dict
    global God_list
    
    curr_bin_sol = get_bin_sol(sol_list)

    if sol_fitness >= best_dict["gbest_fitness"]:
        # get the ai, bi of God_list[Qbest_idx]
        Qgbest_ab = God_list[Qbest_idx]
        # print(f'Get better Gbest: {sol_fitness}!\n')

        # get the binary sol_list to compare
        gbest_bin_sol = get_bin_sol(best_dict["gbest_sol"])

        for var in range(Nvar):
            for bit in range(Nbit):
                # a, b要從尾巴取
                a = round(Qgbest_ab[var][bit][0], 5)
                b = round(Qgbest_ab[var][bit][1], 5)

                if (int(gbest_bin_sol[var][bit]) == 0 and int(curr_bin_sol[var][bit]) == 0):
                    # Let delta_theta = 0.001 * pi
                    delta_theta = theta_2
                    if ((a*b) > 0):
                        rotate(Qbest_idx, var, bit, -delta_theta)

                    elif ((a*b) < 0):
                        rotate(Qbest_idx, var, bit, delta_theta)
                    elif (a == 0 or b == 0):
                        rotate(Qbest_idx, var, bit, delta_theta)

                if (int(gbest_bin_sol[var][bit] == 1) and int(curr_bin_sol[var][bit] == 0)):
                    # Let delta_theta = 0.001 * pi
                    delta_theta = theta_2
                    if (a*b) > 0:
                        rotate(Qbest_idx, var, bit, -delta_theta)
                
                    elif (a*b) < 0:
                        rotate(Qbest_idx, var, bit, delta_theta)

                    elif (a == 0 or b == 0):
                        rotate(Qbest_idx, var, bit, -delta_theta)
                    
                if (int(gbest_bin_sol[var][bit] == 0) and int(curr_bin_sol[var][bit] == 1)):
                    # Let delta_theta = 0.001 * pi
                    delta_theta = theta_2
                    if (a*b) > 0:
                        rotate(Qbest_idx, var, bit, delta_theta)
                        
                    elif (a*b) < 0:
                        rotate(Qbest_idx, var, bit, -delta_theta)
                        
                    elif (a == 0 or b == 0):
                        rotate(Qbest_idx, var, bit, delta_theta)         

                if (int(gbest_bin_sol[var][bit] == 1) and int(curr_bin_sol[var][bit] == 1)):
                    # Let delta_theta = 0.001 * pi
                    delta_theta = theta_2
                    if (a*b) > 0:
                        rotate(Qbest_idx, var, bit, delta_theta)

                    elif (a*b) < 0:
                        rotate(Qbest_idx, var, bit, -delta_theta)

                    elif (a == 0 or b == 0):
                        rotate(Qbest_idx, var, bit, delta_theta)
        # update the best_dict[gbest]
        best_dict["gbest_fitness"] = sol_fitness
        best_dict["gbest_sol"] = sol_list
    
    # if sol_fitness worse than best_dict["gbest_fitness"]
    elif sol_fitness < best_dict["gbest_fitness"]:
        # get the ai, bi of God_list[Qbest_idx]
        Qgbest_ab = God_list[Qbest_idx]

        # get the binary sol_list to compare
        gbest_bin_sol = get_bin_sol(best_dict["gbest_sol"])
        for var in range(Nvar):
            for bit in range(Nbit):
                # a, b要從尾巴取
                a = Qgbest_ab[var][Nbit - 1 - bit][0]
                b = Qgbest_ab[var][Nbit - 1 - bit][1]
                if (int(gbest_bin_sol[var][bit] == 0) and int(curr_bin_sol[var][bit] == 0)):
                    # Let delta_theta = 0.001 * pi
                    delta_theta = theta_2
                    if (a*b) > 0:
                        rotate(Qbest_idx, var, bit, -delta_theta)
    
                    elif (a*b) < 0:
                        rotate(Qbest_idx, var, bit, delta_theta)
                        
                    elif (a == 0 or b == 0):
                        rotate(Qbest_idx, var, bit, delta_theta)

                if int(gbest_bin_sol[var][bit] == 1) and int(curr_bin_sol[var][bit] == 0):
                    # Let delta_theta = 0.08 * pi
                    delta_theta = theta_1
                    if (a*b) > 0:
                        rotate(Qbest_idx, var, bit, -delta_theta)

                    elif (a*b) < 0:
                        rotate(Qbest_idx, var, bit, delta_theta)

                    elif (a.real == 0 or b.real == 0):
                        rotate(Qbest_idx, var, bit, delta_theta)
                    
                if (int(gbest_bin_sol[var][bit]) == 0 and int(curr_bin_sol[var][bit]) == 1):
                    # Let delta_theta = 0.01 * pi
                    delta_theta = theta_1
                    if (a*b) > 0:
                        rotate(Qbest_idx, var, bit, delta_theta)

                    elif (a*b) < 0:
                        rotate(Qbest_idx, var, bit, -delta_theta)
                        
                    elif (a == 0 or b == 0):
                        rotate(Qbest_idx, var, bit, delta_theta)

                if (int(gbest_bin_sol[var][bit]) == 1 and int(curr_bin_sol[var][bit]) == 1):
                    # Let delta_theta = 0.01 * pi
                    delta_theta = theta_2
                    if (a*b) > 0:
                        rotate(Qbest_idx, var, bit, delta_theta)

                    elif (a*b) < 0:
                        rotate(Qbest_idx, var, bit, -delta_theta)
                
                    elif (a.real == 0 or b.real == 0):
                        rotate(Qbest_idx, var, bit, delta_theta)

    # Compare for pbest_solution
    if sol_fitness >= best_dict["pbest_fitness"][sol]:
        # print(f'Get Better pbest sol_{sol}: {sol_fitness}')
        # get the ai, bi of God_list[Qsol]
        Qpbest_ab = God_list[sol]
        # get the binary sol_list to compare
        pbest_bin_sol = get_bin_sol(best_dict["pbest_list"][sol])

        for var in range(Nvar):
            for bit in range(Nbit):
                # a, b要從尾巴取                
                a = round(Qpbest_ab[var][bit][0], 4)
                b = round(Qpbest_ab[var][bit][1], 4)
                if int(pbest_bin_sol[var][bit] == 0) and int(curr_bin_sol[var][bit] == 0):
                    # Let delta_theta = 0.01 * pi
                    delta_theta = theta_2
                    if (a*b) > 0:
                        rotate(sol, var, bit, -delta_theta)

                    elif (a*b) < 0:
                        rotate(sol, var, bit, delta_theta)

                    elif (a == 0 or b == 0):
                        rotate(sol, var, bit, delta_theta)

                if (int(pbest_bin_sol[var][bit] == 1) and int(curr_bin_sol[var][bit] == 0)):
                    # Let delta_theta = 0.01 * pi
                    delta_theta = theta_2
                    if (a*b) > 0:
                        rotate(sol, var, bit, -delta_theta)

                    elif (a*b) < 0:
                        rotate(sol, var, bit, delta_theta)
                        
                    elif (a == 0 or b == 0):
                        rotate(sol, var, bit, delta_theta)
                    
                if (int(pbest_bin_sol[var][bit] == 0) and int(curr_bin_sol[var][bit]) == 1):
                    # Let delta_theta = 0.01 * pi
                    delta_theta = theta_2
                    if (a*b) > 0:
                        rotate(sol, var, bit, delta_theta)

                    elif (a*b) < 0:
                        rotate(sol, var, bit, -delta_theta)

                    elif (a == 0 or b == 0):
                        rotate(sol, var, bit, delta_theta)

                if (int(pbest_bin_sol[var][bit] == 1) and int(curr_bin_sol[var][bit] == 1)):
                    # Let delta_theta = 0.01 * pi
                    delta_theta = theta_2
                    if ((a*b) > 0):
                        rotate(sol, var, bit, delta_theta)

                    elif ((a*b) < 0):
                        rotate(sol, var, bit, -delta_theta)

                    elif ((a*b) == 0):
                        rotate(sol, var, bit, delta_theta)

        best_dict["pbest_fitness"][sol] = sol_fitness
        best_dict["pbest_list"][sol] = sol_list
    
    elif sol_fitness < best_dict["pbest_fitness"][sol]:
        # get the ai, bi of God_list[Qsol]
        Qpbest_ab = God_list[sol]
        # get the binary sol_list to compare
        pbest_bin_sol = get_bin_sol(best_dict["pbest_list"][sol])
        for var in range(Nvar):
            for bit in range(Nbit):
                # a, b要從尾巴取
                a = round(Qpbest_ab[var][Nbit - 1 - bit][0], 4)
                b = round(Qpbest_ab[var][Nbit - 1 - bit][1], 4)
                if (int(pbest_bin_sol[var][bit] == 0) and int(curr_bin_sol[var][bit] == 0)):
                    # Let delta_theta = 0.01 * pi
                    delta_theta = theta_2
                    if (a*b) > 0:
                        rotate(sol, var, bit, -delta_theta)

                    elif (a*b) < 0:
                        rotate(sol, var, bit, delta_theta)

                    elif (a == 0 or b == 0):
                        rotate(sol, var, bit, delta_theta)

                if int(pbest_bin_sol[var][bit] == 1) and int(curr_bin_sol[var][bit] == 0):
                    # Let delta_theta = 0.01 * pi
                    delta_theta = theta_1
                    if (a*b) > 0:
                        rotate(sol, var, bit, -delta_theta)

                    elif (a*b) < 0:
                        rotate(sol, var, bit, delta_theta)

                    elif (a == 0 or b == 0):
                        rotate(sol, var, bit, delta_theta)
                    
                if (int(pbest_bin_sol[var][bit] == 0) and int(curr_bin_sol[var][bit]) == 1):
                    # Let delta_theta = 0.01 * pi
                    delta_theta = theta_1
                    if (a*b) > 0:
                        rotate(sol, var, bit, delta_theta)
                    elif (a*b) < 0:
                        rotate(sol, var, bit, -delta_theta)
                    elif (a == 0 or b == 0):
                        rotate(sol, var, bit, delta_theta)

                if (int(pbest_bin_sol[var][bit]) == 1 and int(curr_bin_sol[var][bit]) == 1):
                    # Let delta_theta = 0.01 * pi
                    delta_theta = theta_2
                    if ((a*b) > 0):
                        rotate(sol, var, bit, delta_theta)
                    elif ((a*b) < 0):
                        rotate(sol, var, bit, -delta_theta)
                    elif ((a*b) == 0):
                        rotate(sol, var, bit, delta_theta)
        Qgbest_ab = (God_list[Qbest_idx])
        print(Qgbest_ab)

    return God_list

In [23]:
def FIT_cal(XX):
    '''
    This function define the OBJ function and return the fitness value.
    In this example we use the square sum of each variable.
    '''
    SUM = 0
    for var in range(Nvar):
        SUM += XX[var] * XX[var]
    return SUM

In [24]:

#################### main function ####################
time_start = time.time()  # count the time
God_list = God_init()
best_dict = best_dict_init()
print(best_dict)
for sol in range(Nsol):
    sol_list = sol_update(sol)
    sol_fitness = FIT_cal(sol_list)
    print(f"Initialize sol_{sol}:", sol_list)
    print("sol_fitness: ", sol_fitness)
    God_update(sol, sol_list, sol_fitness)

for gen in range(Ngen):
    print(f'=============== Generation_{gen} ===============')
    for sol in range(Nsol):
        print(f'----- sol_{sol} -----')
        sol_list = sol_update(sol)
        sol_fitness = FIT_cal(sol_list)
        print('sol_fitness: {}'.format(sol_fitness))
        print('sol_list: ', sol_list)
        God_update(sol, sol_list, sol_fitness)

    print('Global best fitness: {}'.format(best_dict['gbest_fitness']))
    print('Global best solution: {}'.format(best_dict['gbest_sol']))

time_end = time.time()  # 結束計時
time_c = time_end - time_start  # 執行所花時間
print('time cost', time_c, 's')
print('Global best fitness: {}'.format(best_dict['gbest_fitness']))
print('Global best solution: {}'.format(best_dict['gbest_sol']))

#################### main function ####################


{'gbest_fitness': 0, 'gbest_sol': [0, 0, 0, 0, 0, 0], 'pbest_fitness': [0, 0, 0, 0, 0], 'pbest_list': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]}
Initialize sol_0: [86, 549, 791, 899, 332, 272]
sol_fitness:  1926887
Initialize sol_1: [905, 88, 558, 720, 304, 559]
sol_fitness:  2061430
Initialize sol_2: [364, 497, 577, 431, 429, 431]
sol_fitness:  1267997
Initialize sol_3: [225, 400, 878, 556, 812, 541]
sol_fitness:  2242670
Initialize sol_4: [544, 97, 825, 406, 754, 141]
sol_fitness:  1739203
=============== Generation_0 ===============
----- sol_0 -----
sol_fitness: 2336467
sol_list:  [121, 612, 987, 940, 133, 268]
----- sol_1 -----
sol_fitness: 967834
sol_list:  [456, 103, 840, 92, 16, 187]
[[[0.70407615 0.71012448]
  [0.70399827 0.71020169]
  [0.70399827 0.71020169]
  [0.70395933 0.71024029]
  [0.70403721 0.71016308]
  [0.70714555 0.70706801]
  [0.70702923 0.70718432]
  [0.70407615 0.71012448]
  [0.70718432 0.70702923]
  [0.7